In [1]:
import src.operations.reaction_Cl as reaction_Cl

In [2]:
import numpy as np

In [3]:
N = 20000

cellsize = 100

Particle_dtype = np.dtype([
    ('pos', np.float64, (3,)),
    ('vel', np.float64, (3,)),
    ('E', np.float64),
    ('id', np.int32)
], align=True)  # 添加 align=True

Cell_dtype = np.dtype([
    ('id', np.int32),
    ('index', np.int32, (3,)),
    ('film', np.int32, (5,)),
    ('normal', np.float64, (3,))
], align=True)  # 添加 align=True

particle = np.zeros(N, dtype=Particle_dtype)
# cell = np.zeros((cellsize, cellsize, cellsize), dtype=CELLDTYPE)

pos = np.random.rand(N, 3)*99
vel = np.random.rand(N, 3)
energy = np.linalg.norm(vel, axis=1)
vel[:,0] = np.divide(vel[:,0], energy)
vel[:,1] = np.divide(vel[:,1], energy)
vel[:,2] = np.divide(vel[:,2], energy)


# 填充粒子数据
particle['pos'] = pos
particle['vel'] = vel
particle['E'] = 100
particle['id'] = 0
particle[int(N/2):]['id'] = 1
cell = np.zeros((cellsize, cellsize, cellsize), dtype=Cell_dtype)

nn = np.array([0, 0, 1])
# nn = nn/np.linalg.norm(nn)

cell['id'] = 1
cell['index'] = np.array([1,2,3])
cell['film'] = np.array([10,10,10,10,0])
cell['normal'] = nn

cell[:,:,50:]['id'] = 1
print(Particle_dtype.itemsize)
print(Cell_dtype.itemsize)

64
64


In [4]:
film = np.ones((cellsize, cellsize, cellsize, 5), dtype=np.int32)*10

film[:,:,:,-1] = 0

In [13]:
def posvel(parcel, pos, vel, weight, energy, typeID):

    parcelIn = np.zeros((pos.shape[0], 9))
    parcelIn[:, :3] = pos
    parcelIn[:, 3:6] = vel
    parcelIn[:, 6] = weight
    parcelIn[:, 7] = energy
    parcelIn[:, 8] = typeID

    parcel = np.concatenate((parcel, parcelIn))

    return parcel

parcel = np.array([[95, 95, 59, 0, 0, 1, 10, 50, 0]])

In [14]:
parcel = posvel(parcel, pos, vel, 10, 100, 0)

In [15]:
parcel = parcel[1:]
parcel.shape

(20000, 9)

In [16]:
parcel

array([[ 79.32148095,  44.11366057,   0.24017038, ...,  10.        ,
        100.        ,   0.        ],
       [ 79.67026541,   5.0035193 ,  20.13879586, ...,  10.        ,
        100.        ,   0.        ],
       [ 46.52069081,  29.81578113,  25.31397116, ...,  10.        ,
        100.        ,   0.        ],
       ...,
       [ 34.63192351,  95.65815933,  56.50968683, ...,  10.        ,
        100.        ,   0.        ],
       [  4.99749469,  37.13439065,  30.34726354, ...,  10.        ,
        100.        ,   0.        ],
       [ 31.13216533,   7.18143548,  12.66742785, ...,  10.        ,
        100.        ,   0.        ]])

In [9]:
film_label_index_normal = np.zeros((cellsize, cellsize, cellsize, 7))

film_label_index_normal[:,:,:,0] = 1
film_label_index_normal[:,:,:,1:4] = np.array([1,2,3])
film_label_index_normal[:,:,:,4:7] = nn
film_label_index_normal[:,:20,:40,0] = 0

In [10]:
film_label_index_normal[0,0,0]

array([0., 1., 2., 3., 0., 0., 1.])

In [11]:
reaction_Cl.reaction_rate_parallel_all(film, parcel, film_label_index_normal, np.array([cellsize, cellsize, cellsize]))

(array([[[[10, 10, 10, 10,  0],
          [10, 10, 10, 10,  0],
          [10, 10, 10, 10,  0],
          ...,
          [10, 10, 10, 10,  0],
          [10, 10, 10, 10,  0],
          [10, 10, 10, 10,  0]],
 
         [[10, 10, 10, 10,  0],
          [10, 10, 10, 10,  0],
          [10, 10, 10, 10,  0],
          ...,
          [10, 10, 10, 10,  0],
          [10, 10, 10, 10,  0],
          [10, 10, 10, 10,  0]],
 
         [[10, 10, 10, 10,  0],
          [10, 10, 10, 10,  0],
          [10, 10, 10, 10,  0],
          ...,
          [10, 10, 10, 10,  0],
          [10, 10, 10, 10,  0],
          [10, 10, 10, 10,  0]],
 
         ...,
 
         [[10, 10, 10, 10,  0],
          [10, 10, 10, 10,  0],
          [10, 10, 10, 10,  0],
          ...,
          [10, 10, 10, 10,  0],
          [10, 10, 10, 10,  0],
          [10, 10, 10, 10,  0]],
 
         [[10, 10, 10, 10,  0],
          [10, 10, 10, 10,  0],
          [10, 10, 10, 10,  0],
          ...,
          [10, 10, 10, 10,  0],
 

In [17]:
%timeit  -n 1000 reaction_Cl.reaction_rate_parallel_all(film, parcel, film_label_index_normal, np.array([cellsize, cellsize, cellsize]))

756 μs ± 62.2 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [39]:
parcel.shape

(20000, 9)